# Import Library

In [ ]:
import numpy as np
import pandas as pd
import json

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#sastrawi
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#swifter
!pip install swifter

#nltk
import nltk
from nltk.corpus import stopwords

#vis
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Preapare the Data

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### JSON

In [ ]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### CSV

In [ ]:
import pandas as pd

def load_data(file):
    # Read the CSV file into a pandas DataFrame
    data = pd.read_csv(file, sep=';', encoding='utf-8')
    return data

def write_data(file, data):
    # Write the pandas DataFrame to a CSV file
    data.to_csv(file, sep=';', encoding='utf-8', index=False)

# Example usage
# output_file_path = 'drive/MyDrive/Colab Notebooks/Topic Modeling - LDA/Output.csv'
# write_data(output_file_path, data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Load the Data

### JSON

In [ ]:
# # Load data with id/label "texts"
# data = load_data("data/ushmm_dn.json")["texts"]

# print (data[0][0:90])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### CSV

In [ ]:
file_path = 'drive/MyDrive/Colab Notebooks/Topic Modeling - LDA/Test.csv'
data = pd.read_csv(file_path, sep=';', encoding='utf-8')

data.head(1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,No,Text
0,1,Kita harus bersukur di tahun 2020 sampai 2030 ...


# Data Pre-Processing

### Case Folding (LowerText)

In [ ]:
data['Text'] = data['Text'].str.lower()

print(data['Text'].head(1))

0    kita harus bersukur di tahun 2020 sampai 2030 ...
Name: Text, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Clean Data

In [ ]:
nltk.download('punkt')

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data['Text'] = data['Text'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['Text'] = data['Text'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data['Text'] = data['Text'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data['Text'] = data['Text'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data['Text'] = data['Text'].apply(remove_singl_char)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:23: DeprecationWarning: invalid escape sequence '\s'
<>:23: DeprecationWarning: invalid escape sequence '\s'
<ipython-input-40-92da03f71dbf>:23: DeprecationWarning: invalid escape sequence '\s'
  return re.sub('\s+',' ',text)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Tokenizing

In [ ]:
# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['Text_tokens'] = data['Text'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(data['Text_tokens'].head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Tokenizing Result : 

0    [kita, harus, bersukur, di, tahun, sampai, nan...
Name: Text_tokens, dtype: object


In [ ]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

data['textdata_tokens_fdist'] = data['Text_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(data['textdata_tokens_fdist'].head().apply(lambda x : x.most_common()))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Frequency Tokens : 

0    [(saya, 8), (anak, 6), (kita, 5), (ini, 5), (s...
Name: textdata_tokens_fdist, dtype: object


### Stopwords

In [ ]:
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'bisnis', 'pandemi', 'indonesia',
                       'saya', 'sekali', 'ini', 'harus', 'sempat', 'terima', 'kasih',
                       'untuk'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['textdata_tokens_WSW'] = data['Text_tokens'].apply(stopwords_removal)

print(data['textdata_tokens_WSW'].head())

0    [bersukur, bonus, demografi, penduduk, usia, p...
Name: textdata_tokens_WSW, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Lemmatization/Stemming

In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['textdata_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))

47


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

    # untuk melihat hasilnya silahkan jalankan baris di bawah ini
print(term,":" ,term_dict[term])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tercapai : capai


In [ ]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['textdata_tokens_stemmed'] = data['textdata_tokens_WSW'].swifter.apply(get_stemmed_term)

print(data['textdata_tokens_stemmed'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

0    [bersukur, bonus, demografi, duduk, usia, prod...
Name: textdata_tokens_stemmed, dtype: object


# LDA Modul Using Gensim

In [ ]:
doc_clean = data['textdata_tokens_stemmed']
doc_clean

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    [bersukur, bonus, demografi, duduk, usia, prod...
Name: textdata_tokens_stemmed, dtype: object

In [ ]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary<47 unique tokens: ['amas', 'anak', 'antil', 'bangga', 'berani']...>


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 2 # jumlah topik yang akan di extract
number_words = 5 # jumlah kata per topik

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Running and Trainign LDA model on the document term matrix.
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)
lda_model.show_topics(num_topics=total_topics, num_words=number_words)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.021*"anak" + 0.021*"muda" + 0.021*"sempat" + 0.021*"amas" + 0.021*"senang"'),
 (1,
  '0.073*"anak" + 0.051*"muda" + 0.051*"sempat" + 0.040*"amas" + 0.028*"kerja"')]

In [ ]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])
print(df_imp_wcount)

       word  topic_id  importance  word_count
0      anak         0    0.021279           6
1      muda         0    0.021279           4
2    sempat         0    0.021279           4
3      amas         0    0.021278           3
4    senang         0    0.021278           2
5      ucap         0    0.021278           2
6     orang         0    0.021278           2
7     teman         0    0.021278           2
8     kerja         0    0.021278           2
9      buka         0    0.021276           1
10     anak         1    0.073489           6
11     muda         1    0.050872           4
12   sempat         1    0.050872           4
13     amas         1    0.039563           3
14    kerja         1    0.028254           2
15    orang         1    0.028254           2
16    teman         1    0.028254           2
17     ucap         1    0.028254           2
18   senang         1    0.028254           2
19  prabowo         1    0.016946           1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Define the path to save the JSON file in Google Drive
output_json_file_path = '/content/drive/My Drive/Colab Notebooks/Topic Modeling - LDA/lda_model_word_counts.json'

# Save the DataFrame to JSON
df_imp_wcount.to_json(output_json_file_path, orient='records')

print("Word counts of topic keywords saved to Google Drive:", output_json_file_path)

Word counts of topic keywords saved to Google Drive: /content/drive/My Drive/Colab Notebooks/Topic Modeling - LDA/lda_model_word_counts.json


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def format_topics_sentences(ldamodel=None, corpus=None, texts=None):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                new_row = pd.DataFrame([[int(topic_num), round(prop_topic, 4), topic_keywords]], columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])
                sent_topics_df = pd.concat([sent_topics_df, new_row], ignore_index=True)
            else:
                break

    # Add original text to the end of the output
    contents = pd.Series(texts, name='Text')
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Assuming lda_model, doc_term_matrix, and doc_clean are already defined
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index(drop=True)

# The DataFrame already has the correct columns, no need to reset them
df_dominant_topic.columns = ['Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

print(df_dominant_topic.head(10))

   Dominant_Topic  Topic_Perc_Contrib  \
0               1              0.9913   

                                            Keywords  \
0  anak, muda, sempat, amas, kerja, orang, teman,...   

                                                Text  
0  [bersukur, bonus, demografi, duduk, usia, prod...  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Visualization

In [ ]:
import pyLDAvis.gensim
import pickle
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))

corpus = [dictionary.doc2bow(text) for text in doc_clean]

if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.save_html(LDAvis_prepared, 'drive/MyDrive/Colab Notebooks/Topic Modeling - LDA/ldavis_prepared_'+ str(total_topics) +'.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
LDAvis_prepared

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.010579  0.0       1        1  99.131017
0     -0.010579  0.0       2        1   0.868983, topic_info=      Term      Freq     Total Category  logprob  loglift
26    muda  3.000000  3.000000  Default  30.0000  30.0000
39  sempat  3.000000  3.000000  Default  29.0000  29.0000
0     amas  2.000000  2.000000  Default  28.0000  28.0000
1     anak  4.000000  4.000000  Default  27.0000  27.0000
40  senang  1.000000  1.000000  Default  26.0000  26.0000
..     ...       ...       ...      ...      ...      ...
40  senang  0.012019  1.832572   Topic2  -3.8501  -0.2814
44    ucap  0.012019  1.832572   Topic2  -3.8501  -0.2814
29   orang  0.012019  1.832572   Topic2  -3.8501  -0.2814
42   teman  0.012019  1.832572   Topic2  -3.8501  -0.2814
20   kerja  0.012019  1.832572   Topic2  -3.8501  -0.2814

[102 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         1  1.171301           amas
1         1  1.053226           anak
2         1  0.905869          antil
3         1  0.905869         bangga
4         1  0.905868         berani
5         1  0.905869       berdukar
6         1  0.905869       bersukur
7         1  0.905869          bonus
8         1  0.905869           buka
9         1  0.905869          capai
10        1  0.905869      demografi
11        1  0.905869          duduk
12        1  0.905869         dukung
13        1  0.905869          fokus
14        1  0.905868        ganding
15        1  0.905869            gus
16        1  0.905869          hebat
17        1  0.905868         hormat
18        1  0.905868          jalan
19        1  0.905869          keras
20        1  1.091362          kerja
21        1  0.905869          ketua
22        1  0.905868     kontestasi
23        1  0.905868          labor
24        1  0.905869         lompat
25        1  0.905869         mahfud
26        1  0.911871           muda
27        1  0.905869       muhaymin
28        1  0.905869       nasional
29        1  1.091362          orang
30        1  0.905868       panggung
31        1  0.905868         partai
32        1  0.905869        peluang
33        1  0.905869          pikir
34        1  0.905868        prabowo
35        1  0.905868      produktif
36        1  0.905868  produktifitas
37        1  0.905869           prof
38        1  0.905869       profesor
39        1  0.911871         sempat
40        1  1.091363         senang
41        1  0.905869         tangan
42        1  1.091362          teman
43        1  0.905868        tingkat
44        1  1.091363           ucap
45        1  0.905869          ulang
46        1  0.905869           usia, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [ ]:
# Only export the value of words (i.e., 'Keywords') to JSON
keywords_json = df_dominant_topic[['Keywords']].to_json(orient='records')

# Save the JSON output to a file in Google Drive
output_json_file_path = 'drive/MyDrive/Colab Notebooks/Topic Modeling - LDA/keywords.json'
with open(output_json_file_path, 'w') as f:
    f.write(keywords_json)

print("Keywords saved to Google Drive:", output_json_file_path)

Keywords saved to Google Drive: drive/MyDrive/Colab Notebooks/Topic Modeling - LDA/keywords.json


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(keywords_json)

[{"Keywords":"anak, muda, sempat, amas, kerja, orang, teman, ucap, senang, prabowo"}]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
